#### Libraries


In [1]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import torch
import timm
import csv
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import os
import itertools
from print_color import print
import time

# 3D to 2D


#### MIP


In [2]:
def mip(image: np.ndarray)->np.ndarray:
    """Create the maximum intensity projection.
    """
    maximum_intensity_projection = np.max(image, axis=0)
    maximum_intensity_projection = np.clip(maximum_intensity_projection, -100, 300)
    
    return maximum_intensity_projection

#### AIP


In [3]:
def aip(image: np.ndarray)->np.ndarray:
    """Create a mean image projection.
    """
    average_intensity_projection = np.mean(image,axis=0)
    average_intensity_projection = np.clip(average_intensity_projection,-100,300)
    
    return average_intensity_projection

#### SLICE


In [4]:
def find_tumor_center(mask: np.ndarray)->int:
    """Find the center of the tumor. 
    """
    return np.clip(np.argmax(np.sum(mask, axis=(1, 2))),-100,300)

In [5]:
def slice_tc(image: np.ndarray, mask: np.ndarray)->np.ndarray:
    """Take a slice at the center of the tumor.
    """
    x1 = find_tumor_center(mask)
    return np.clip(image[x1, :, :],-100,300)

#### TUMOR MIP

In [6]:
def tumor_mip(image: np.ndarray, mask: np.ndarray)->np.ndarray:
    """Return the mip of the tumor only.
    """
    return np.clip(np.max(image * mask, axis=0),-100,300)

#### MINIP

In [7]:
def minip(image: np.ndarray)->np.ndarray:
    """Create the maximum intensity projection.
    """
    return np.clip(np.min(image, axis=0),-100,300)

# Convert the images


In [8]:

MIP = []
AIP = []
SLICE = []
TUMOR_MIP = []
MINIP = []

with open('c:/users/sven/Documents/BEP_sbierenbroodspot_1334859/Pyradiomics/v5/folderNames2.csv', newline='') as file:
    reader = csv.reader(file)
    next(reader)
    
    for row in reader:
        img_data = sitk.GetArrayFromImage(sitk.ReadImage(row[1]))

        if os.path.exists(row[2]):
            mask_data = sitk.GetArrayFromImage(sitk.ReadImage(row[2]))
        else:
            mask_l = sitk.GetArrayFromImage(sitk.ReadImage(row[3]))
            mask_r = sitk.GetArrayFromImage(sitk.ReadImage(row[4]))
            mask_data = mask_l+mask_r

        MIP.append(mip(img_data))
        AIP.append(aip(img_data))
        SLICE.append(slice_tc(img_data,mask_data))
        TUMOR_MIP.append(tumor_mip(img_data,mask_data))
        MINIP.append(minip(img_data))

# feature extraction


In [9]:
def IAC(layout: list, i: int) -> np.ndarray:
    """
    Create the 512x512x3 input array for feature extraction.
    """
    slots = [MIP[i], AIP[i], SLICE[i], TUMOR_MIP[i], MINIP[i]]
    input_array = np.dstack([slots[layout[j] - 1] for j in range(3)])
    return input_array

#### Model


In [11]:
models = ['resnest50d', 'efficientnet_b4', 'gluon_resnet152_v1s','convnext_nano','convnext_large','resnetv2_101','efficientnetv2_rw_m']


#### Feature extraction single image

In [12]:
def feature_extraction(input_array: np.ndarray)-> np.ndarray:
    """Extract the features of a single image.
    """ 
    
    # Convert numpy array to torch tensor and normalize
    input_tensor = torch.from_numpy(np.transpose(input_array, (2, 0, 1))).float()
    input_tensor = input_tensor.to('cuda')
    input_tensor = (input_tensor + 100) / (300 + 100)
    input_tensor = input_tensor.unsqueeze(0)

    # Extract features using ResNet50
    with torch.no_grad():
        features = model.forward_features(input_tensor)
        features = torch.mean(features, dim=[2, 3])

    # Convert features to numpy array
    features = features.to('cpu')
    features_array = features.numpy()
    
    return features_array

#### Feature extraction full dataset

In [13]:
def feature_extraction_full(layout: list, sizing) -> np.ndarray:
    """Extract all features of the dataset for an input layout.
    """
    sizes = [2048, 1792, 2048, 640, 1536, 2048, 2152]
    result = np.empty((0, sizes[sizing]))

    for i in range(len(MIP)):
        result_vector = feature_extraction(IAC(layout, i))
        result = np.vstack([result, result_vector])    
    return result

#### Extract manual result


In [37]:
result  = feature_extraction_full([3, 3, 3], 0)
print(svm_auc(result, labels))
#np.savetxt("result.csv", result, delimiter=",")



ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 32, 1, 1])

# Labels


In [14]:
labels = np.empty((0,0))
with open('c:/users/sven/Documents/BEP_sbierenbroodspot_1334859/Technical_research/Labels.csv', newline='') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        labels = np.append(labels, row[0])

# SVM


In [15]:
def svm_auc(data: np.ndarray, labels: np.ndarray)->np.float64:
    """Get the AUC for a svm.
    """

    # Create a stratified 5-fold cross-validation object
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

    # Initialize an SVM classifier
    svm = SVC(kernel='linear', probability=True, random_state = 2)

    # Store the AUC scores
    auc_scores = []

    # Perform cross-validation
    for train_idx, test_idx in cv.split(data, labels):
        # Split the data and labels
        X_train, X_test = data[train_idx], data[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]

        # Train the SVM classifier
        svm.fit(X_train, y_train)

        #Predict the probabilities for the test set
        
        y_pred_proba = svm.predict_proba(X_test)[:, 1]

        # Compute the AUC score
        auc_score = roc_auc_score(y_test, y_pred_proba)

        # Store the AUC score
        auc_scores.append(auc_score)

    return np.mean(auc_scores), np.std(auc_scores)

# Get all results

In [39]:
auc_total = []
stds_total = []


for n in range(7):

    model = timm.create_model(models[n], pretrained=True)
    model.eval()
    model.to('cuda')
    auc = []
    stds = []
    
    for i in [1,2,4,5]:
        for j in [1,2,4,5]:
            for k in [1,2,4,5]:

                my_list = [i, j, k]
                auc_ = svm_auc(feature_extraction_full(my_list, n),labels)
                auc.append(auc_[0])
                stds.append(auc_[1])
    
    print(np.max(auc))
    auc_total.append(auc)
    stds_total.append(stds)



0.7442424242424244
0.8181818181818181
0.8270707070707071
0.8377777777777778
0.7282828282828282
0.7222222222222222
0.7836363636363636


In [41]:
np.savetxt("aucs.csv", np.transpose(auc_total), delimiter=",") 
np.savetxt("stds.csv", np.transpose(stds_total), delimiter=",") 

In [16]:
auc_total = []
stds_total = []


for n in range(7):

    model = timm.create_model(models[n], pretrained=True)
    model.eval()
    model.to('cuda')
    auc = []
    stds = []
    
    auc_ = svm_auc(feature_extraction_full([3,3,3], n),labels)
    auc.append(auc_[0])
    stds.append(auc_[1])
    
    print(np.max(auc))
    print(np.max(stds))
    auc_total.append(auc)
    stds_total.append(stds)
    

0.49676767676767686
0.0788432853613209
0.5
0.15777376836303011
0.4675757575757576
0.1484031933798124
0.608989898989899
0.07891623835597793
0.48434343434343424
0.11987052158349658
0.5898989898989898
0.07580268017559731
0.44727272727272727
0.0847748598318165


# String generation

In [ ]:
with open('my_file.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for i in range(1, 6):
        for j in range(1, 6):
            for k in range(1, 6):
               
                my_list = [i, j, k]
                    
                csvwriter.writerow([str(my_list)])